In [6]:
#Importing / Shaping dataset
import numpy as np
import tensorflow as tf
(training_inputs, training_results), (test_inputs, test_data) = tf.keras.datasets.mnist.load_data()


In [7]:
print(training_results[0])

5


In [8]:

def vectorized_result(j):
    #One Hot Encoding
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

training_inputs = [np.reshape(x, (784, 1)) / 255 for x in training_inputs]
training_results = [vectorized_result(y) for y in training_results]

test_inputs = [np.reshape(x, (784, 1)) / 255 for x in test_inputs]
test_results = [vectorized_result(y) for y in test_data]

training_data = list(zip(training_inputs, training_results))
test_data = list(zip(test_inputs, test_results))



In [10]:
print(training_results[0])

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]]


In [3]:
class Network(object):
    #The constructor
    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        #if there is test_data, then n_test is the number of elements in test_data
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
            np.random.shuffle(training_data)
            #produces mini batches of size mini_batch_size
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {:} / {:} / {:}".format(j, self.evaluate(test_data), n_test))
            else:
                print("Epoch{:} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        for x, y in mini_batch:
            #We can see that backprop() takes inputs x, y which have shape (784, 1) and (10, 1) respectively.
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.biases = [b - (eta/len(mini_batch)) * nb for b, nb in zip(self.biases, nabla_b)]
        self.weights = [w - (eta/len(mini_batch)) * nw for w, nw in zip(self.weights, nabla_w)]

    def backprop(self, x, y):
        #Each element of this list is a vector. Its length is the number of nodes, which is just the number of biases.
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        #feedforward
        activation = x
        #Stores all the activations, layer by layer
        activations = [x]
        zs = []
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)

        #backward pass
        #Calculate delta for the last layer, L
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].T)

        #We already did the case where l = 1. That's just the last layer. Now we are considering l = 2 through l = self.num_layers - 1
        #Does not evaluate the case where l = self.num_layers
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].T, delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].T)
        return(nabla_b, nabla_w)

    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), np.argmax(y)) for (x,y) in test_data]
        return sum(int(x == y) for (x,y) in test_results)

    def cost_derivative(self, output_activations, y):
            return (output_activations - y)

def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z) * (1-sigmoid(z))


In [4]:
net = Network([784, 50, 100, 10])

net.SGD(training_data, 30, 10, 3.0, test_data = test_data)

Epoch 0 / 7485 / 10000
Epoch 1 / 7608 / 10000
Epoch 2 / 7675 / 10000
Epoch 3 / 8470 / 10000
Epoch 4 / 8576 / 10000
Epoch 5 / 8554 / 10000
Epoch 6 / 8620 / 10000
Epoch 7 / 8611 / 10000
Epoch 8 / 8628 / 10000
Epoch 9 / 8639 / 10000
Epoch 10 / 8639 / 10000
Epoch 11 / 8665 / 10000
Epoch 12 / 8689 / 10000
Epoch 13 / 8654 / 10000
Epoch 14 / 8669 / 10000
Epoch 15 / 8703 / 10000
Epoch 16 / 8692 / 10000
Epoch 17 / 8698 / 10000
Epoch 18 / 8712 / 10000
Epoch 19 / 8707 / 10000
Epoch 20 / 8689 / 10000
Epoch 21 / 8568 / 10000
Epoch 22 / 8678 / 10000
Epoch 23 / 8708 / 10000
Epoch 24 / 8707 / 10000
Epoch 25 / 8707 / 10000
Epoch 26 / 8707 / 10000
Epoch 27 / 8735 / 10000
Epoch 28 / 8711 / 10000
Epoch 29 / 8682 / 10000
